# Carla speed limit assist with corrective actions and traffic light detection and warning

## Import library
- **cv2 (OpenCV):** computer vision library for image processing. Used for handling and processing video frames or images.
- **paho.mqtt.client:** client library for the MQTT protocol, this library permit to publish MQTT topics, likely for exchanging data between the CARLA simulator and other systems.
- **ultralytics (YOLO):** provides Python interface for the YOLO (You Only Look Once) object detection framework. It provide object detection tasks, such as identify traffic light, speedLimit.
- **pygame:** library for creating games and multimedia applications, it allows to take input events.

In [ ]:
import glob
import os
import sys
import cv2
import numpy as np
import paho.mqtt.client as mqtt
from ultralytics import YOLO
import time
import pygame

try:
    sys.path.append(glob.glob('../../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

## Broker MQTT configuration

In [ ]:
BROKER = "localhost"
PORT = 1883    

clientMQTT = mqtt.Client()
clientMQTT.connect(BROKER, PORT, 60)
def sendEventToBroker(topic, message):
    try:
        clientMQTT.publish(topic, message)
    except Exception as e:
        pass

## Client configuration

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
client.load_world('Town01')
world = client.get_world()
spectator = world.get_spectator()

## Helpful functions used in this notebook
- **spawn_vehicle:** Adds a vehicle to the simulation.
  - Picks a vehicle type from the blueprint library (pattern).
  - Selects a spawn point on the map (spawn_index).
  - Places the vehicle at that location.
- **spawn_camera** Adds a camera to the simulation.
  - Sets the camera size, field of view (foV), and update speed.
  - Places the camera at a specific position and angle.
  - Attaches the camera to a vehicle.



In [ ]:
IMAGE_SIZE = 640
currect_map = False

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

if world.get_map().name == "Town02" or world.get_map().name == "Town01":
    currect_map = True

def spawn_camera(attach_to=None, sensor_tick=0, transform=carla.Transform(carla.Location(x=0.7, z=1.8), carla.Rotation(pitch=5, yaw=35)), width=IMAGE_SIZE, height=IMAGE_SIZE, foV=50, exposure_mode='auto', expousure_compensation=0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('fov', str(foV))
    camera_bp.set_attribute('sensor_tick', str(sensor_tick))
    camera_bp.set_attribute('bloom_intensity', "0")
    camera_bp.set_attribute('exposure_mode', exposure_mode)
    camera_bp.set_attribute('exposure_compensation', str(expousure_compensation))  # Riduce la luminosità
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

## Training YOLOv8 Models for Object Detection

### Overview
We have trained the **YOLOv8** model multiple times to detect specific objects with two categories:

1. **Speed Limits**:
    - **First Training**: Using images of the speed limit signs without any background.
    - **Second Training**: Using images of speed limit signs in real-world environments, with significant background clutter for object detection.

2. **Traffic Lights**:
    - **First Training**: Using images of the traffic lights without any background.
    - **Second Training**: Using images of traffic lights in real-world environments, with significant background clutter for object detection.

At the end, we will have **two YOLOv8 models**:
- One dedicated to **speed limits**.
- One dedicated to **traffic lights**.

### Dataset
The datasets used for training were sourced from the following site:  
[Roboflow Universe](https://universe.roboflow.com/)

### Base Code for Downloading and Training the Model
Below is the base code to download the dataset and train the YOLOv8 model:


In [ ]:
# !pip install roboflow
# from roboflow import Roboflow
# rf = Roboflow(api_key="...")
# project = rf.workspace("wawan-pradana").project("cinta_v2")
# dataset = project.version(1).download("yolov8")
# model = YOLO('yolov8n.pt')
# results = model.train(data="data.yaml", epochs=80, imgsz=416, save=True)

## Training Model Results
<img src="runsSpeed/detect/train/confusion_matrix.png" alt="Descrizione dell'immagine" width="700">
<img src="runsTrafficLight/detect/train3/confusion_matrix.png" alt="Descrizione dell'immagine" width="700">





## Load Pretrained Models

In [ ]:
model = YOLO("runsSpeed/detect/train/weights/best.pt")
#modelTrafficLight = YOLO("runsUlt/detect/train7/weights/best.pt")
modelTrafficLight = YOLO("runsFinLight/detect/train/weights/best.pt")
modelTrafficLightOver = YOLO("runsFinLight/detect/train/weights/best.pt")
#modelTrafficLightOver = YOLO("runsTrafficLight/detect/train3/weights/best.pt")

## Set weather conditions
- Set the worst weather condition or the best weather condition

In [ ]:
weather = world.get_weather()

In [ ]:
weather.sun_azimuth_angle = 0
weather.sun_altitude_angle = -90
weather.cloudiness = 100
weather.precipitation = 100
weather.precipitation_deposits = 100
weather.wind_intensity = 100
weather.fog_density = 100
weather.fog_distance = 1 

In [ ]:
weather.sun_azimuth_angle = 0 
weather.sun_altitude_angle = 90
weather.cloudiness = 20
weather.precipitation = 0
weather.precipitation_deposits = 0
weather.wind_intensity = 0
weather.fog_density = 30
weather.fog_distance = 100

In [ ]:
world.set_weather(weather)

## Utility functions
- **colorSpeedLimit**: Change the letters color in the view.
- **calcColor**: Change the letters color of the traffic light

In [ ]:
def colorSpeedLimit(current_speed, speed_limit):
    if speed_limit == None or int(current_speed) <= int(speed_limit):
        return (0, 255, 0) 
    else:
        return (0, 0, 255)

In [ ]:
def calcColor():
    if traffic_light == "red":
        return (0, 0, 255)
    elif traffic_light == "yellow":
        return (0, 255, 255)
    elif traffic_light == "green":
        return (0, 255, 0)
    else:
        return (255, 255, 255)

## Camera_callback:

This function processes an image captured by a camera, analyzes it using a machine learning model, and identifies the most likely speed limit sign if found with sufficient confidence:

1. **Global Variables:**
   - `last_analysis_time`: Tracks the time of the last analysis to ensure periodic processing.
   - `last_speed_limit`: Stores the most recently detected speed limit.

2. **Image Data Conversion:**
   - The raw image data is read into a NumPy array using `np.frombuffer`, which converts the data into a byte array.
   - The image is reshaped into its original dimensions, including the alpha channel (RGBA format).
   - The alpha channel is removed to work with the RGB image, creating `image_bgr`.

3. **Periodic Analysis:**
   - Checks if enough time `analysis_interval` has passed since the last analysis using the current timestamp.
   - If the time interval condition is met, updates `last_analysis_time`.

4. **Model Prediction:**
   - The resized image is passed to a machine learning model (`model`) for analysis.
   - Extracts the detected class IDs (`class_ids`) and their confidence scores (`confidences`) from the model's results.

5. **Identify Speed Limit:**
   - Iterates through the detected objects, finding the class ID with the highest confidence score.
   - If the confidence score exceeds a threshold (`0.87`), determines the corresponding class name.

6. **Extract Speed Limit and Notify:**
   - Attempts to extract the speed limit value from the class name.
   - Updates `last_speed_limit` with the detected value.
   - Sends an event with the detected speed limit to an external broker using `sendEventToBroker`.


In [ ]:
def camera_callback(image):
    global last_speed_limit

    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3]  

    results = model(image_bgr, verbose=False)
    limit_id = None
    confidence_max = 0
    class_ids = results[0].boxes.cls.numpy()
    confidences = results[0].boxes.conf.numpy()
    
    for class_id, confidence in zip(class_ids, confidences):
        if confidence > confidence_max:
            limit_id = class_id
            confidence_max = confidence

    if confidence_max > 0.87:
        class_name = class_names[int(limit_id)]
        try:
            last_speed_limit = class_name.split(" ")[2]
            sendEventToBroker("speedLimit", "Detected " + last_speed_limit)
        except:
            pass

## Camera_view_callback:

This function processes an image captured by a camera, calculates the vehicle's speed, and updates the global variables used for video output and speed monitoring:

1. **Global Variables:**
   - `video_output`: A global variable used to store the processed image data for visualization or further use.
   - `speed_car`: A global variable used to store the vehicle's current speed.

2. **Speed Calculation:**
   - Retrieves the vehicle's velocity using `vehicle.get_velocity()`, which provides the velocity components along the X, Y, and Z axes.
   - Computes the magnitude of the velocity vector using the formula for Euclidean norm.
   - Converts the speed from meters per second (m/s) to kilometers per hour (km/h) by multiplying by 3.6.
   - Updates the global variable `speed_car` with the calculated speed.


In [ ]:
def camera_view_callback(image):
    global video_output, speed_car
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    velocity_car = vehicle.get_velocity()
    speed_car = 3.6 * (velocity_car.x**2 + velocity_car.y**2 + velocity_car.z**2)**0.5
    video_output = image_np

## Camera_traffic_callback:
This function processes an image from a camera to detect the current state of a traffic light, applying enhancements to the image and analyzing it with a machine learning model. It updates global variables and notifies changes:

1. **Global Variables:**
   - `last_analysis_time_trafficLight`: Tracks the time of the last traffic light analysis to ensure periodic processing.
   - `traffic_light`: Stores the detected traffic light state (`"red"`, `"yellow"`, `"green"`, or `"Not detected"`).

2. **Image Data Conversion:**
   - Converts the raw image data from the camera into a NumPy array using `np.frombuffer`.
   - Reshapes the array into an image matrix of size `IMAGE_SIZE x IMAGE_SIZE` with 4 channels (BGRA format).
   - Creates a copy of the RGB portion (`image_bgr`) for further processing.

3. **Image Preprocessing:**
   - **Color Enhancement:** Increases the red channel values to accentuate red tones, reduces blue and green values slightly to suppress their dominance, and creates an enhanced image for better detection.
   - **Clipping Values:** Ensures pixel values remain in the valid range (0–255).

1. **Periodic Analysis:**
   - Checks if the interval since the last analysis exceeds a defined threshold (`analysis_interval_trafficLight`).
   - Updates `last_analysis_time_trafficLight` when a new analysis is performed.

2. **Traffic Light Detection:**
   - Passes the processed image (`resized_image`) to a traffic light detection model (`modelTrafficLight`).
   - Retrieves the detected class IDs (`class_ids`) and their confidence scores (`confidences`).
   - Handles cases where no traffic lights are detected by setting `traffic_light` to `"Not detected"`.

3. **Identify Most Likely Traffic Light:**
   - Iterates through detected objects, identifying the class ID with the highest confidence score.
   - It assigns the detected class name as the current traffic light state.
   - Applies confidence thresholds:
     - High confidence (`>0.63`) for red light.
     - Lower confidence (`>0.47`) for other colors.

4. **Update and Notify:** Sends an event to an external broker using `sendEventToBroker` with the detected traffic light state.

In [ ]:
def camera_traffic_callback(image):
    global traffic_light#, video_output2
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3].copy()
    resized_image = image_bgr.copy()
    #video_output2 = image_bgr

    results = modelTrafficLight(resized_image, verbose=False)
    trafficLight_id = None
    confidence_max = 0
    class_ids = results[0].boxes.cls.numpy()
    confidences = results[0].boxes.conf.numpy()

    if class_ids.size == 0:
        traffic_light = "Not detected"
    else:
        for class_id, confidence in zip(class_ids, confidences):
            if confidence > confidence_max:
                trafficLight_id = class_id
                confidence_max = confidence

        if traffic_light != None and class_names_trafficLight[int(trafficLight_id)] == "yellow" and traffic_light == "red":
            class_name = "red"
        else:
            class_name = class_names_trafficLight[int(trafficLight_id)]
        if confidence_max > 0.55 and class_name=="red" or confidence_max > 0.50 and class_name=="green" or confidence_max > 0.50 and class_name=="yellow":
            # print(confidence_max)
            if traffic_light != class_name:
                traffic_light = class_name
                sendEventToBroker("TrafficLight", "Detected trafficlight " + class_name)

## Camera_traffic_red_callback:

This function operates similarly to `camera_traffic_callback(image)` but focuses specifically on detecting and reacting to **red traffic lights**, modifying a global flag (`red_over`) based on the detection results:

1. **Global Variables:**
   - `last_analysis_time_trafficLight_red`: Tracks the time of the last analysis for red traffic lights to ensure periodic updates.
   - `red_over`: A flag indicating whether a red light condition has been detected and action of stopping should be enforced.

2. **Traffic Light Detection:**
   - Passes the processed image to the `modelTrafficLight`, which detects traffic light states and outputs class IDs (`class_ids`) and their confidence scores (`confidences`).
   - Handles cases where no traffic lights are detected by resetting `red_over` to `False`.

3. **Red Light Detection Logic:**
   - Iterates through detected objects, identifying the class ID with the highest confidence score.
   - If the detected class is `"red"` and the confidence score exceeds `0.65`: Sets `red_over` to `True` **unless the vehicle is in reverse (`gear != "Reverse"`)** or another condition prevents it.
   - If the detected class is `"green"` or `"yellow"` and the confidence score exceeds `0.43`: Resets `red_over` to `False`, allowing movement.

In [ ]:
def camera_traffic_red_callback(image):
    global red_over
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3].copy()
    resized_image = image_bgr.copy()

    results = modelTrafficLightOver(resized_image, verbose=False)

    trafficLight_id = None
    confidence_max = 0

    class_ids = results[0].boxes.cls.numpy()
    confidences = results[0].boxes.conf.numpy()

    if class_ids.size != 0:
        for class_id, confidence in zip(class_ids, confidences):
            if confidence > confidence_max:
                trafficLight_id = class_id
                confidence_max = confidence

        class_name = class_names_trafficLight[int(trafficLight_id)]
        if class_name=="red" and confidence_max > 0.50:
            if(gear != "Reverse" and not red_over):
                red_over = True
        elif (class_name=="green" or class_name=="yellow") and confidence_max > 0.43:
            red_over = False
    else:
        red_over = False

## Explanation of the Code

This code use camera feeds, traffic light detection, and manual/automatic controls:

### 1. **Setup and Initialization**
   - **Pygame Window:** Creates a minimal interface using Pygame, used for event handling keyboard inputs.
   - **Global Variables:**
     - Includes control parameters for throttle, brake, and steering.
     - Flags for managing speed control, red light detection, and traffic light states.
     - Constants for proportional control of throttle and brake adjustments based on speed error.
   - **Timers and Analysis Intervals:** Tracks the timing of camera analyses for speed, traffic lights, and red light violations.
   - **Camera Feeds:** Initializes multiple cameras:
     - Main RGB camera for capturing the environment.
     - Additional cameras for detecting traffic lights, speed limits and specific red light conditions.
   - **Vehicle Spawning:** Spawns a simulated vehicle in CARLA and attaches the cameras to it.

### 2. **Real-Time Display and Feedback**
   - Displays:
     - Current speed and detected speed limits.
     - Traffic light state (e.g., `"red"`, `"green"`).
     - A warning if the vehicle violates a red light.

### 3. **Camera-Based Analysis**
   - Each camera captures frames periodically, which are processed for specific tasks:
     - **Speed Limit Detection:** Uses object detection to identify speed limit signs.
     - **Traffic Light State Detection:** Identifies traffic light states and updates global flags.
     - **Red Light Violations:** Specifically monitors red lights and determines if the vehicle violates them.

### 4. **Event Handling and Controls**
   - **Keyboard Controls:**
     - Toggle speed control (`E`), red light detection (`O`), and autopilot (enable `C`, disable `V`).
     - Manual acceleration: (`W`), braking (`S`), steering (`A`/`D`), and gear selection (`R`/`F`).
   - **Speed Control Logic:**
     - Automatically adjusts throttle and brake based on the speed limit.
     - Includes a "dead zone" around the speed limit to prevent constant adjustments.
   - **Red Light Response:**
     - Automatically engages brakes when a red light is detected and speed under 40 km/h.

### 5. **Vehicle State Updates**
   - Continuously updates the vehicle's control inputs (`throttle`, `brake`, `steer`) based on manual or automatic inputs.
   - Ensures smooth transitions, such as gradual steering return to center when no input is applied.

### 6. **Cleanup**
   - Gracefully terminates the simulation, destroying the cameras, vehicle, and display, and disconnecting any external connections (e.g., MQTT broker).


In [ ]:
# Initialize a small Pygame display for handling events
display = pygame.display.set_mode((50, 50))
pygame.display.set_caption("Manual Control CARLA")

# Define control variables and increments
steer_increment = 0.02  # Increment for steering adjustments
throttle_increment = 0.1  # Increment for throttle adjustments
steer = 0.0  # Current steering value
throttle = 0.0  # Current throttle value
brake = 0.0  # Current brake value
global gear  # Define a global gear variable
gear = "Drive"  # Initial gear state
speed_control_activate = False  # Flag for speed control activation
red_over_activate = False  # Flag for red light detection activation
red_over = False  # Red light violation flag

# Constants for proportional controller
KP_THROTTLE = 0.15  # Proportional gain for acceleration
KP_BRAKE = 0.02  # Proportional gain for braking
DEAD_ZONE = 3.0  # Dead zone around the speed limit
MIN_THROTTLE = 0.2  # Minimum throttle value
MIN_BRAKE = 0.1  # Minimum brake value

# Initialize the output video frame
video_output = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)

# Variables for speed and traffic light tracking
last_speed_limit = None  # Last detected speed limit
speed_car = 0  # Current vehicle speed
traffic_light = "Not detected"  # Current traffic light state

# Load class names for object detection models
class_names = model.names  # Object detection model names
class_names_trafficLight = modelTrafficLight.names  # Traffic light model names

# Spawn the simulated vehicle
vehicle = spawn_vehicle()


# Attach and configure cameras for various purposes
camera = spawn_camera(attach_to=vehicle, sensor_tick=0.2)  # Main camera
camera.listen(lambda image: camera_callback(image))  # Listen for incoming frames

camera_view = spawn_camera(
    attach_to=vehicle, 
    sensor_tick=0.0,
    transform=carla.Transform(carla.Location(x=1, z=1.5), carla.Rotation(pitch=0, yaw=0)), 
    width=IMAGE_SIZE, 
    height=IMAGE_SIZE, 
    foV=70
)  # Driver's perspective camera
camera_view.listen(lambda image: camera_view_callback(image))

# Additional cameras for traffic light and red light detection
if currect_map:
    camera_trafficLight = spawn_camera(
        attach_to=vehicle, 
        sensor_tick=0.35,
        transform=carla.Transform(carla.Location(x=0.3, y=1, z=1.2), carla.Rotation(pitch=5, yaw=0)), 
        width=IMAGE_SIZE, 
        height=IMAGE_SIZE, 
        foV=40,
    )
    camera_trafficLight.listen(lambda image: camera_traffic_callback(image))
    
    camera_trafficLight_red_detector = spawn_camera(
        attach_to=vehicle,
        sensor_tick=0.35,
        transform=carla.Transform(carla.Location(x=-1.8, y=0, z=2), carla.Rotation(pitch=0, yaw=0)), 
        width=IMAGE_SIZE, 
        height=IMAGE_SIZE, 
        foV=110
    )
    camera_trafficLight_red_detector.listen(lambda image: camera_traffic_red_callback(image))

# Create an OpenCV window for displaying results
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

#cv2.namedWindow('RGB Camera2', cv2.WINDOW_AUTOSIZE)

try:
    clock = pygame.time.Clock()  # Pygame clock for controlling loop rate
    event_timer = 0  # Timer for event processing
    EVENT_RATE = 100  # Event handling interval in milliseconds

    while True:
        # Add text overlays to the output frame
        temp_frame = video_output.copy()
        cv2.putText(
            temp_frame,
            f"Last Speed Limit: {last_speed_limit}",
            (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        cv2.putText(
            temp_frame,
            f"Current speed: {speed_car:.0f}",
            (10, 45),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            colorSpeedLimit(speed_car, last_speed_limit),
            2,
            cv2.LINE_AA,
        )

        # Display traffic light information and warnings
        if currect_map:
            if red_over:
                val = "You ran a red light"
                sendEventToBroker("TrafficLightViolation", "Red light violation")
            else:
                val = ""
            cv2.putText(
                temp_frame,
                val,
                (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 255),
                2,
                cv2.LINE_AA,
            )
        cv2.putText(
            temp_frame,
            f"Traffic light: {traffic_light}",
            (10, 70),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            calcColor(),
            2,
            cv2.LINE_AA,
        )

        # Show the frame with overlays
        cv2.imshow('RGB Camera', temp_frame)
        #cv2.imshow('RGB Camera2', video_output2.copy())

        # Break the loop on 'q' key press
        if cv2.waitKey(1) == ord('q'):
            break

        # Handle Pygame events periodically
        current_time = pygame.time.get_ticks()
        if current_time - event_timer > EVENT_RATE:
            event_timer = current_time
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    break
                elif event.type == pygame.KEYUP:  # Handle key releases
                    if event.key == pygame.K_e:  # Toggle speed control
                        speed_control_activate = not speed_control_activate
                    if event.key == pygame.K_c:  # Enable autopilot
                        vehicle.set_autopilot(True)
                    if event.key == pygame.K_v:  # Disable autopilot
                        vehicle.set_autopilot(False)
                    if event.key == pygame.K_o:  # Toggle red light detection
                        red_over_activate = not red_over_activate

            keys = pygame.key.get_pressed()  # Get pressed keys

            # Handle speed control logic
            if speed_control_activate and last_speed_limit and keys[pygame.K_w]:
                # Calculate speed error relative to the limit
                error = float(last_speed_limit) - float(speed_car) + 3
                if error < -DEAD_ZONE:
                    throttle = 0
                    brake = max(KP_BRAKE * abs(error), MIN_BRAKE)
                else:
                    throttle = max(KP_THROTTLE * error, MIN_THROTTLE)
                    brake = 0
            elif keys[pygame.K_w]:  # Manual throttle control
                throttle = min(throttle + throttle_increment, 1)
                brake = 0
            else:
                throttle = 0
                brake = 0
            
            if keys[pygame.K_s]:  # Manual brake control
                brake = min(brake + throttle_increment * 4, 1)
                throttle = 0

            if red_over and int(speed_car) <= 40 and gear != "Reverse" and red_over_activate:
                brake = 1
                throttle = 0

            # Handle steering controls
            if keys[pygame.K_r]:  # Reverse gear
                gear = "Reverse"
                red_over = False
            elif keys[pygame.K_f]:  # Forward gear
                gear = "Drive"
            elif keys[pygame.K_a]:  # Steer left
                steer = max(steer - steer_increment, -1)
            elif keys[pygame.K_d]:  # Steer right
                steer = min(steer + steer_increment, 1)
            else:
                steer = steer * 0.9  # Gradually return to center

            # Apply vehicle controls
            control = carla.VehicleControl()
            control.throttle = throttle
            control.brake = brake
            control.steer = steer
            control.reverse = (gear == "Reverse")
            vehicle.apply_control(control)

        clock.tick(40)  # Maintain a 40 FPS loop rate
finally:
    # Clean up resources and connections
    cv2.destroyAllWindows()
    pygame.quit()
    camera.destroy()
    camera_trafficLight.destroy()
    camera_trafficLight_red_detector.destroy()
    camera_view.destroy()
    vehicle.destroy()
    clientMQTT.disconnect()
